# Generate Clickstream Data
**This notebook generates clickstream data with explicit frequent viewing of products before placing an order.**

**THIS NOTEBOOK CAN BE RUN IN PARALLEL WITH `1_setup.ipynb`**

**Recommended settings to run this notebook in SageMaker Studio:**

- Image: Data Science
- Kernel: Python3
- Instance type: <font color='blue'>ml.m5.large (2 vCPU + 8 GiB)</font>

---

## Contents

1. [Background](#Background)

## Background


This is for tracking viewing habits and translating them to useful conversion figures.

## Setup 

### Prerequisites

In [1]:
!pip install Faker confluent-kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 126.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 133.9 MB/s eta 0:00:00


#### Imports

In [2]:
from botocore.client import ClientError
from collections import defaultdict
from faker import Faker
import pandas as pd
import numpy as np
import sagemaker
import datetime
import hashlib
import random
import boto3
import math
import os

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


#### Seed for Reproducibility

In [5]:
faker = Faker()
faker.seed_locale('en_US', 0)
SEED = 123
random.seed(SEED)
np.random.seed(SEED)
faker.seed_instance(SEED)

#### Constants

In [ ]:
import json
import time
import random
from faker import Faker
from confluent_kafka import Producer

# Initialize Faker and Kafka Producer
fake = Faker()
# Possible event types
event_types = ['page_view', 'click', 'add_to_cart', 'purchase']

def delivery_report(err, msg):
    """Called once for each message produced to indicate delivery result."""
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}]')

def generate_dummy_event():
    """Generates a dummy customer interaction event."""
    return {
        'event_id': fake.uuid4(),
        'timestamp': fake.iso8601(),
        'customer_id': random.randint(1, 1000),
        'session_id': fake.uuid4(),
        'event_type': random.choice(event_types),
        'product_id': random.randint(1, 500),
        'product_category': random.choice(['electronics', 'fashion', 'home', 'books', 'toys']),
        'price': round(random.uniform(10.0, 500.0), 2)
    }

#### Simple generation of random events with no clear directions towards a purchase

In [7]:
import json
import time
import random
import datetime
from faker import Faker
from confluent_kafka import Producer

# Initialize Faker and Kafka Producer
fake = Faker()
# Define extended event types including scrolling behavior
event_types = ['page_view', 'scroll', 'click', 'add_to_cart', 'purchase']
def generate_session_events(customer_id):
    """Generates an ordered list of dummy events for a given session."""
    session_id = fake.uuid4()
    event_chain = []
    # Start with a random timestamp (e.g., within the current month)
    current_time = fake.date_time_this_month().strftime('%Y-%m-%d %H:%M:%S')
    # Random number of events for the session (between 5 and 10 events)
    n_events = random.randint(5, 10)
    
    for i in range(n_events):
        # Randomly choose an event type
        event_type = random.choice(event_types)
        # Increment time by a random delta (5 to 30 seconds) to simulate order
        current_dt = datetime.datetime.strptime(current_time, '%Y-%m-%d %H:%M:%S')
        delta = datetime.timedelta(seconds=random.randint(5, 30))
        current_dt += delta
        current_time = current_dt.strftime('%Y-%m-%d %H:%M:%S')
        
        event = {
            'event_id': fake.uuid4(),
            'timestamp': current_time,
            'customer_id': customer_id,
            'session_id': session_id,
            'event_type': event_type,
            'product_id': random.randint(1, 500),
            'product_category': random.choice(['electronics', 'fashion', 'home', 'books', 'toys']),
            'price': round(random.uniform(10.0, 500.0), 2),
            'order_in_session': i + 1  # Indicates the order of the event within the session
        }
        event_chain.append(event)
    return event_chain

# Continuously generate and send session events
currentSessions, totalSessions = 0, 10
allSessions = []
while currentSessions < totalSessions:
    # Pick a random customer
    customer_id = random.randint(1, 1000)
    session_events = generate_session_events(customer_id)
    allSessions.append(session_events)
    currentSessions += 1

In [103]:
print([len(sess) for sess in allSessions])
for event in allSessions[1][:]:
    print(event)
    print("---------------------------")

[6, 7, 10, 5, 9, 8, 6, 7, 10, 9]
{'event_id': '575ded00-8dd0-4300-a7e6-7818bb1150b5', 'timestamp': '2025-03-04 03:19:39', 'customer_id': 472, 'session_id': '6fe11191-4fea-42da-afc9-e2d5389adb88', 'event_type': 'purchase', 'product_id': 9, 'product_category': 'books', 'price': 265.23, 'order_in_session': 1}
---------------------------
{'event_id': '5e12d07f-de9d-48fd-b635-9e6124f6dd68', 'timestamp': '2025-03-04 03:19:55', 'customer_id': 472, 'session_id': '6fe11191-4fea-42da-afc9-e2d5389adb88', 'event_type': 'click', 'product_id': 110, 'product_category': 'books', 'price': 468.94, 'order_in_session': 2}
---------------------------
{'event_id': '388d26f5-a35a-4770-be1d-0b72ec7f03b9', 'timestamp': '2025-03-04 03:20:16', 'customer_id': 472, 'session_id': '6fe11191-4fea-42da-afc9-e2d5389adb88', 'event_type': 'page_view', 'product_id': 425, 'product_category': 'electronics', 'price': 300.48, 'order_in_session': 3}
---------------------------
{'event_id': '24b68581-3110-4c51-83f2-e5b8a03876c0

#### Generation of  events with clear directions towards a purchase

Based on these factors, users will indicate higher chances of an actual purchase:
- moderate dwell times
- 2 or more consecutive interactions of page clicks, views, scrolling

In [115]:
import json
import time
import random
import datetime
from faker import Faker
from confluent_kafka import Producer

# Initialize Faker and Kafka Producer
fake = Faker()

# Define a dwell time distribution (in seconds) as (min, max, probability)
dwell_distribution = [
    (1, 3, 0.05),    # 10% of events: very quick interactions
    (3, 10, 0.70),   # 60% of events: typical dwell times
    (10, 30, 0.20),  # 20% of events: longer engagement
    (30, 60, 0.05)   # 10% of events: very long dwell times
]

def get_dwell_time():
    """Return a dwell time sampled from the defined distribution."""
    r = random.random()
    cumulative = 0
    for start, end, prob in dwell_distribution:
        cumulative += prob
        if r <= cumulative:
            return round(random.uniform(start, end), 2)
    return round(random.uniform(3, 10), 2)

def generate_session_events(customer_id):
    """
    Simulate a user session on a well-functioning site.
    
    - The session consists of one or more product chains.
    - Within each chain, a single product is focused on.
    - The user starts with a page_view and performs a series of events (page_view, scroll, click).
    - The chance to add to cart increases when:
        • There are multiple consecutive interactions (e.g. clicks),
        • And the dwell time between events is moderate.
    - At the end of the session, if any product was added to cart, a purchase event is generated.
    """
    session_id = fake.uuid4()
    events = []
    current_time = fake.date_time_this_month().strftime('%Y-%m-%d %H:%M:%S')
    order_in_session = 1
    cart = []  # Track product_ids added to cart

    # Decide the number of product chains in the session (e.g., 1 to 3)
    num_chains = random.randint(1, 10)
    
    for _ in range(num_chains):
        # For each chain, choose a single product to focus on
        product_id = random.randint(1, 500)
        product_category = random.choice(['electronics', 'fashion', 'home', 'books', 'toys'])
        consecutive_interactions = 0
        chain_length = random.randint(3, 6)
        
        for i in range(chain_length):
            dwell = get_dwell_time()
            # Update the timestamp based on the dwell time
            current_dt = datetime.datetime.strptime(current_time, '%Y-%m-%d %H:%M:%S')
            current_dt += datetime.timedelta(seconds=dwell)
            current_time = current_dt.strftime('%Y-%m-%d %H:%M:%S')
            
            # First event in chain is always a page_view
            if i == 0:
                event_type = 'page_view'
            else:
                # Choose randomly among browsing events
                event_type = random.choice(['page_view', 'scroll', 'click'])
                if event_type == 'click':
                    consecutive_interactions += 1
                else:
                    # Reduce consecutive interaction count slightly for non-clicks
                    consecutive_interactions = max(0, consecutive_interactions - 1)
            
            # Increase chance for add_to_cart if conditions are met
            if (i > 1 and product_id not in cart and consecutive_interactions >= 2 and 3 <= dwell <= 10):
                if random.random() < 0.7:
                    event_type = 'add_to_cart'
                    cart.append(product_id)
            
            event = {
                'event_id': fake.uuid4(),
                'timestamp': current_time,
                'customer_id': customer_id,
                'session_id': session_id,
                'event_type': event_type,
                'product_id': product_id,
                'product_category': product_category,
                'price': round(random.uniform(10.0, 500.0), 2),
                'order_in_session': order_in_session
            }
            events.append(event)
            order_in_session += 1

    # At the end of the session, if any products were added to the cart, generate a purchase event.
    if cart:
        dwell = get_dwell_time()
        current_dt = datetime.datetime.strptime(current_time, '%Y-%m-%d %H:%M:%S')
        current_dt += datetime.timedelta(seconds=dwell)
        current_time = current_dt.strftime('%Y-%m-%d %H:%M:%S')
        purchase_event = {
            'event_id': fake.uuid4(),
            'timestamp': current_time,
            'customer_id': customer_id,
            'session_id': session_id,
            'event_type': 'purchase',
            'purchased_items': cart,  # List of product_ids
            'total_amount': sum(round(random.uniform(10.0, 500.0), 2) for _ in cart),
            'order_in_session': order_in_session
        }
        events.append(purchase_event)
        order_in_session += 1

    # Optionally, the user may continue browsing after purchase (simulate one additional chain)
    if random.random() < 0.5:
        product_id = random.randint(1, 500)
        product_category = random.choice(['electronics', 'fashion', 'home', 'books', 'toys'])
        chain_length = random.randint(2, 4)
        for _ in range(chain_length):
            dwell = get_dwell_time()
            current_dt = datetime.datetime.strptime(current_time, '%Y-%m-%d %H:%M:%S')
            current_dt += datetime.timedelta(seconds=dwell)
            current_time = current_dt.strftime('%Y-%m-%d %H:%M:%S')
            event = {
                'event_id': fake.uuid4(),
                'timestamp': current_time,
                'customer_id': customer_id,
                'session_id': session_id,
                'event_type': random.choice(['page_view', 'scroll']),
                'product_id': product_id,
                'product_category': product_category,
                'price': round(random.uniform(10.0, 500.0), 2),
                'order_in_session': order_in_session
            }
            events.append(event)
            order_in_session += 1

    return events

# # Continuously generate and send session events to Kafka
# while True:
#     customer_id = random.randint(1, 1000)
#     session_events = generate_session_events(customer_id)
#     for event in session_events:
#         producer.produce(topic, json.dumps(event).encode('utf-8'), callback=delivery_report)
#         producer.poll(0)
#         time.sleep(random.uniform(0.5, 1.5))
#     time.sleep(random.uniform(2, 5))

currentSessions, totalSessions = 0, 50
allSessions3 = []
# Continuously generate and send session events
while currentSessions < totalSessions:
    # Pick a random customer
    customer_id = random.randint(1, 1000)
    session_events = generate_session_events(customer_id)
    allSessions3.extend(session_events)
    currentSessions += 1

In [116]:
print(len(allSessions3))
# for event in allSessions3[:10]:
#     print(event)
#     print("---------------------------")
df_all = pd.DataFrame(allSessions3)
df_all.head()

1286


,event_id,timestamp,customer_id,session_id,event_type,product_id,product_category,price,order_in_session,purchased_items,total_amount
0,5eae5744-01c2-4197-841b-4c2eabb1af0d,2025-03-04 05:03:22,291,3c9da555-1317-4f5a-a0a0-a5e9794f1656,page_view,471.0,electronics,389.78,1,NaN,NaN
1,48d26657-2d94-4422-938e-5dae3dd88029,2025-03-04 05:03:31,291,3c9da555-1317-4f5a-a0a0-a5e9794f1656,scroll,471.0,electronics,490.05,2,NaN,NaN
2,f2bd2870-8d1c-4903-8486-6033039f42dc,2025-03-04 05:03:38,291,3c9da555-1317-4f5a-a0a0-a5e9794f1656,page_view,471.0,electronics,121.06,3,NaN,NaN
3,ff2fd1cf-c3f6-44e0-9d30-be84e8038fc9,2025-03-04 05:03:47,291,3c9da555-1317-4f5a-a0a0-a5e9794f1656,page_view,471.0,electronics,261.25,4,NaN,NaN
4,93260fde-fa4d-42b0-af7a-b20b29455cb6,2025-03-04 05:03:50,291,3c9da555-1317-4f5a-a0a0-a5e9794f1656,page_view,236.0,home,488.74,5,NaN,NaN


In [129]:
int_vals = {
    "purchase": 0,
    "page_view": 4,
    "scroll": 2,
    "click": 1,
    "add_to_cart": 6
}
df_all["interaction_value"] = df_all["event_type"].map(int_vals)

In [131]:
print(len(df_all[df_all["event_type"] == "purchase"]))
df_all[df_all["event_type"] == "click"]

22


,event_id,timestamp,customer_id,session_id,event_type,product_id,product_category,price,order_in_session,purchased_items,total_amount,interaction_value
7,f6e66a7d-e2bc-4cae-8584-0004a12a9ded,2025-03-04 05:04:03,291,3c9da555-1317-4f5a-a0a0-a5e9794f1656,click,236.0,home,67.17,8,NaN,NaN,1
8,721ebe98-b131-4ad8-ac5a-c56e6523a872,2025-03-04 05:04:23,291,3c9da555-1317-4f5a-a0a0-a5e9794f1656,click,236.0,home,327.37,9,NaN,NaN,1
9,fd86816c-252b-4551-90ab-f4b0cb5166c3,2025-03-04 05:04:38,291,3c9da555-1317-4f5a-a0a0-a5e9794f1656,click,236.0,home,344.20,10,NaN,NaN,1
11,ef45cab4-81fe-42b2-a3db-a11ff4f0f180,2025-03-04 05:04:47,291,3c9da555-1317-4f5a-a0a0-a5e9794f1656,click,184.0,toys,383.94,12,NaN,NaN,1
14,69cc8e09-f326-4fce-b008-c55c8b1cedfb,2025-03-04 05:05:31,291,3c9da555-1317-4f5a-a0a0-a5e9794f1656,click,3.0,fashion,392.13,15,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1256,715fb11b-535c-489f-9d9c-c6f98ef57220,2025-03-03 13:51:36,487,f7a2546a-b4f5-45f2-8b50-d24fa750ebc6,click,420.0,home,115.13,34,NaN,NaN,1
1264,3bf044eb-8891-4868-823b-5611ff6d5c74,2025-03-01 15:33:31,299,3243ca60-4889-4e4e-a42a-2a53953391bf,click,432.0,fashion,115.38,4,NaN,NaN,1
1265,3baa55d5-ef5a-47c3-8c51-902a57e01d5a,2025-03-01 15:33:55,299,3243ca60-4889-4e4e-a42a-2a53953391bf,click,432.0,fashion,208.50,5,NaN,NaN,1
1269,4f7ef9a1-0dd4-4176-b7a4-c7c5e545e72c,2025-03-01 15:34:33,299,3243ca60-4889-4e4e-a42a-2a53953391bf,click,267.0,home,394.45,9,NaN,NaN,1


In [126]:
df_grouped = df_all.groupby(["customer_id"])
df_grouped.agg({'event_type': ','.join}) 

,event_type
customer_id,
15,"page_view,scroll,click,page_view,click,scroll,..."
94,"page_view,page_view,click,page_view,scroll,pag..."
103,"page_view,click,click,page_view,page_view,page..."
132,"page_view,page_view,page_view,scroll,page_view..."
216,"page_view,scroll,scroll,page_view,page_view,pa..."
258,"page_view,page_view,click,scroll,click,scroll,..."
259,"page_view,click,click,scroll,page_view,page_vi..."
266,"page_view,scroll,scroll,scroll,page_view,click..."
275,"page_view,click,page_view,page_view,scroll,scr..."


In [133]:
df_all['cumsum_interactions'] = df_all.groupby(['customer_id'])['interaction_value'].cumsum()

#view updated DataFrame
df_all

,event_id,timestamp,customer_id,session_id,event_type,product_id,product_category,price,order_in_session,purchased_items,total_amount,interaction_value,cumsum_interactions
0,5eae5744-01c2-4197-841b-4c2eabb1af0d,2025-03-04 05:03:22,291,3c9da555-1317-4f5a-a0a0-a5e9794f1656,page_view,471.0,electronics,389.78,1,NaN,NaN,4,4
1,48d26657-2d94-4422-938e-5dae3dd88029,2025-03-04 05:03:31,291,3c9da555-1317-4f5a-a0a0-a5e9794f1656,scroll,471.0,electronics,490.05,2,NaN,NaN,2,6
2,f2bd2870-8d1c-4903-8486-6033039f42dc,2025-03-04 05:03:38,291,3c9da555-1317-4f5a-a0a0-a5e9794f1656,page_view,471.0,electronics,121.06,3,NaN,NaN,4,10
3,ff2fd1cf-c3f6-44e0-9d30-be84e8038fc9,2025-03-04 05:03:47,291,3c9da555-1317-4f5a-a0a0-a5e9794f1656,page_view,471.0,electronics,261.25,4,NaN,NaN,4,14
4,93260fde-fa4d-42b0-af7a-b20b29455cb6,2025-03-04 05:03:50,291,3c9da555-1317-4f5a-a0a0-a5e9794f1656,page_view,236.0,home,488.74,5,NaN,NaN,4,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,5f28d73b-d2ad-4a6f-a290-c6dcba04a833,2025-03-01 15:36:44,299,3243ca60-4889-4e4e-a42a-2a53953391bf,purchase,NaN,NaN,NaN,21,[25],493.87,0,60
1282,0ab7492c-1331-4f92-a3a1-f48d9a6cfe90,2025-03-01 15:36:49,299,3243ca60-4889-4e4e-a42a-2a53953391bf,page_view,365.0,electronics,103.72,22,NaN,NaN,4,64
1283,325824dc-f2ed-4992-aaeb-9ee5c1909a9c,2025-03-01 15:36:53,299,3243ca60-4889-4e4e-a42a-2a53953391bf,page_view,365.0,electronics,100.75,23,NaN,NaN,4,68
1284,1e8781cd-e151-4b75-b9eb-ad3f84c28b78,2025-03-01 15:37:00,299,3243ca60-4889-4e4e-a42a-2a53953391bf,page_view,365.0,electronics,368.17,24,NaN,NaN,4,72


In [134]:
df_all[df_all["event_type"] == "purchase"]

,event_id,timestamp,customer_id,session_id,event_type,product_id,product_category,price,order_in_session,purchased_items,total_amount,interaction_value,cumsum_interactions
25,fe18d764-a598-491e-a432-738a4dd41117,2025-03-04 05:07:18,291,3c9da555-1317-4f5a-a0a0-a5e9794f1656,purchase,NaN,NaN,NaN,26,[3],86.04,0,71
130,e18c0699-9e2d-4388-a8bc-c8106171a8f0,2025-03-03 17:26:34,980,5286faa1-6853-47a4-b3a0-570395f4462f,purchase,NaN,NaN,NaN,38,"[363, 375, 74]",725.71,0,109
187,b8273f56-d400-42c5-b560-d6b9916dc630,2025-03-04 01:22:38,858,9c6c2e21-9a33-4eb5-8558-43e054bdaba2,purchase,NaN,NaN,NaN,22,[117],78.09,0,65
237,530f1485-f3d4-4081-99c8-df43952c095b,2025-03-04 00:17:15,357,1a9f9acf-42fb-47a3-bd8c-de25fd5d230f,purchase,NaN,NaN,NaN,50,"[132, 299]",576.53,0,137
261,848c0e40-c5e9-4b5b-8d3b-43f358d26c24,2025-03-01 01:17:05,833,c2fba6b9-c0f2-4574-bbef-ac6cc021a8fc,purchase,NaN,NaN,NaN,22,[482],131.08,0,57
323,d1308378-b4f1-4797-ada5-ce402aa0252c,2025-03-03 04:06:32,470,0257dfa8-1879-4dd9-8f30-0179b7f8f014,purchase,NaN,NaN,NaN,32,[129],295.90,0,99
382,e47544c1-096f-405f-89f6-7e2716d5639c,2025-03-04 04:29:22,669,7a1fb574-0bfa-4758-97e6-a940248c2f6d,purchase,NaN,NaN,NaN,20,"[333, 15]",609.42,0,60
448,7143b5f1-16d6-4950-b981-43a06474e8c5,2025-03-03 04:59:01,512,626551ff-9b71-4167-8691-48a96fe29e73,purchase,NaN,NaN,NaN,23,[392],148.24,0,63
500,63b5d27b-c0f8-4eb0-b7ce-8d9eb914b202,2025-03-02 11:19:54,283,dd5b3930-3577-4721-bfcf-79adfa3e3b56,purchase,NaN,NaN,NaN,28,"[85, 277]",119.51,0,77
509,2b547019-3030-4b77-9660-00489e7ccb04,2025-03-03 00:42:15,560,001b8f2a-c327-4e7b-8ff1-e216c9645223,purchase,NaN,NaN,NaN,9,[200],111.96,0,25
